# Workshop MQTT - IoT
## Taller práctico de MQTT aplicado a IoT

### Objetivo del Taller
Ofrecer a los participantes una comprensión fundamental de los protocolos de comunicación en el Internet de las Cosas (IoT), con un enfoque práctico en MQTT. Al finalizar, los asistentes podrán entender la arquitectura de MQTT, sus componentes principales y cómo implementar una comunicación básica entre dispositivos utilizando Python.

## 1. Preparación del Taller 

### 1.1. Ejecución del MQTT Broker

Utilizaremos la herramienta Docker para desplegar un broker MQTT localmente. Si no tienes Docker instalado, puedes descargarlo desde [aquí](https://www.docker.com/get-started).

Las siguientes son algunas opciones de brokers MQTT que puedes utilizar:

- [Eclipse Mosquitto](https://mosquitto.org/): Un broker MQTT de código abierto y ligero.
- [HiveMQ](https://www.hivemq.com/): Un broker MQTT comercial que ofrece una versión comunitaria.
- [EMQX](https://www.emqx.io/): Un broker MQTT escalable y de alto rendimiento.
- [NanoMQ](https://nanomq.io/): Un broker MQTT ultraligero diseñado para dispositivos IoT.

Para esta taller utilizaremos NanoMQ:

In [ ]:
# Ejecuta el siguiente comando para iniciar un contenedor Docker con NanoMQ

!docker run -it -d --rm -p 1883:1883 --name mqtt-broker nanomq:latest-slim

Con el docker instalado, puedes probar la conexión utilizando el programa MQTT Explorer, disponible en [aquí](https://mqtt-explorer.com/).

Configura la conexión al broker con los siguientes parámetros:

- Host: `localhost`
- Puerto: `1883`
- Usuario: (dejar en blanco) o `admin`
- Contraseña: (dejar en blanco) o `admin`

![mqtt-explorer](./images/image_001.png)

### 1.2. Preparación del Entorno Python

Para este taller ocupamos algunas dependencias de Python, para trabajar con el protocolo MQTT. Esas dependencias están listadas en el archivo `requirements.txt`. Para instalarlas todas las dependencias dejaremos que el comando `pip` lea el archivo e instale todo lo necesario.

Ejecuta el siguiente bloque.

In [ ]:
%pip install -r requirements.txt

## 2. Manos a la obra

### 2.1. Enviar un mensaje a MQTT

El siguiente bloque de código envía un mensaje al broker MQTT en el topic `iot/workshop/sensor/sala1/temperatura`. Puedes ejecutar el bloque varias veces para enviar múltiples mensajes. Recuerda mantener el MQTT Explorer abierto para ver los mensajes que se publican.

En el MQTT Explorer, te puedes suscribir al topic `iot/workshop/sensor/sala1/#` para ver todos los mensajes publicados en los subtopics de `sala1`, el caracter `#` es un comodín que indica "todos los subtopics".

In [ ]:
# Importación de la librería Paho-MQTT
import paho.mqtt.client as mqtt
import time

# --- Configuración del Cliente MQTT ---
BROKER_ADDRESS = "127.0.0.1"
PORT = 1883
QOS_LEVEL = 1  # 0 = At most once, 1 = At least once, 2 = Exactly once
TOPIC = "iot/workshop/sensor/sala1/temperatura"

# Inicialización del cliente MQTT, en versión MQTT 3.1.1
client = mqtt.Client(protocol=mqtt.MQTTv311)

# Conexión al broker MQTT
client.connect(BROKER_ADDRESS, PORT, 60)

# Definición del mensaje, en este caso usamos un mensaje simple de texto, puedes variar este texto
mensaje = f"Temperatura: 23.5°C\nTS: {time.time()}"

# Publicación del mensaje en el topic especificado con el nivel de QoS definido
client.publish(TOPIC, mensaje, qos=QOS_LEVEL)

# Desconectar el cliente MQTT
client.disconnect()

Debido a que en IoT es común que el contenido del mensaje sea utilizado para análisis posteriores, es recomendable utilizar un formato estructurado como JSON para los mensajes. Esto facilita la interpretación y el procesamiento de los datos en sistemas posteriores, por ejemplo, podemos cambiar el ejemplo anterior a lo siguiente:

In [ ]:
# Importación de la librería Paho-MQTT
import paho.mqtt.client as mqtt
import time
import json

# --- Configuración del Cliente MQTT ---
BROKER_ADDRESS = "127.0.0.1"
PORT = 1883
QOS_LEVEL = 1  # 0 = At most once, 1 = At least once, 2 = Exactly once
TOPIC = "iot/workshop/sensor/sala1/temperatura"

# Inicialización del cliente MQTT, en versión MQTT 3.1.1
client = mqtt.Client(protocol=mqtt.MQTTv311)

# Conexión al broker MQTT
client.connect(BROKER_ADDRESS, PORT, 60)

# Definición del mensaje, en este caso lo definimos como un diccionario y luego lo convertimos a JSON
mensaje = {
    "temperatura": 23.5,
    "unidad": "°C",
    "timestamp": time.time()
}

# Publicación del mensaje en el topic especificado con el nivel de QoS definido
client.publish(TOPIC, json.dumps(mensaje), qos=QOS_LEVEL)

# Desconectar el cliente MQTT
client.disconnect()